In [1]:
#Setting some initial params here
image_size = (224, 224)
save_dir = f"{image_size}"

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torch.cuda.amp import GradScaler, autocast

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

num_epochs = 30
batch_size = 50
learning_rate = 0.001

# we need to normalize
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = ImageFolder(root=r"C:\Users\computer\Downloads\cosoco\dataset\train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=30, pin_memory=True)

classes = train_dataset.classes
print("Detected classes:", classes)

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)

scaler = GradScaler()

os.makedirs(save_dir, exist_ok=True)

print("\nStarting training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_batches = len(train_loader)

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

        # Print every 10 batches
        if (i + 1) % 10 == 0 or (i + 1) == total_batches:
            avg_loss = running_loss / (i + 1)
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{total_batches}], Average Loss: {avg_loss:.4f}")

    print(f"Epoch [{epoch+1}/{num_epochs}] finished. Average Loss: {running_loss / total_batches:.4f}")

    # Save the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        save_path = os.path.join(save_dir, f"kernel_model_weights_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"Model saved to {save_path}")

final_save_path = os.path.join(save_dir, f"kernel_model_weights_epoch_{num_epochs}.pth")
torch.save(model.state_dict(), final_save_path)
print(f"Final model saved to {final_save_path}")

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_dataset = ImageFolder(root=r"C:\Users\computer\Downloads\cosoco\dataset\test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=30, pin_memory=True)
classes = test_dataset.classes

# Evaluate models from epoch 5 to 30 inclusive
for epoch in range(5, 16, 5):
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride = 1, padding = 1, bias = False)
    model.load_state_dict(torch.load(f"{save_dir}\\kernel_model_weights_epoch_{epoch}.pth", map_location=device))
    model.to(device)
    model.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100.0 * correct / total
    print(f"Accuracy of model at epoch {epoch}: {accuracy:.2f}%")


In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import numpy as np
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# transform to match resnet18 mean/std
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dir = r"C:\Users\computer\Downloads\cosoco\dataset\train"
test_dir = r"C:\Users\computer\Downloads\cosoco\dataset\test"
train_set = ImageFolder(root=train_dir, transform=transform)
test_set = ImageFolder(root=test_dir, transform=transform)
train_loader = DataLoader(train_set, batch_size=80, shuffle=True, num_workers=15)
test_loader = DataLoader(test_set, batch_size=80, shuffle=False, num_workers=15)

state_dict = torch.load(f"{save_dir}\kernel_model_weights_epoch_10.pth", map_location=device)

# remove the info from the last layer
state_dict.pop('fc.weight', None)
state_dict.pop('fc.bias', None)

model = models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

model.load_state_dict(state_dict, strict=False)

# remove last layer so we can do feature extraction
model = torch.nn.Sequential(*(list(model.children())[:-1]))
model.to(device)
model.eval()

train_features = []
train_labels = []

test_features = []
test_labels = []

with torch.no_grad():
    for imgs, lbls in tqdm(train_loader, desc="Extracting train features"):
        imgs = imgs.to(device)
        feats = model(imgs).squeeze()
        train_features.append(feats.cpu().numpy())
        train_labels.extend(lbls.numpy())

    for imgs, lbls in tqdm(test_loader, desc="Extracting test features"):
        imgs = imgs.to(device)
        feats = model(imgs).squeeze()
        test_features.append(feats.cpu().numpy())
        test_labels.extend(lbls.numpy())

train_features = np.vstack(train_features)
train_labels = np.array(train_labels)

test_features = np.vstack(test_features)
test_labels = np.array(test_labels)

print("Train feature matrix shape:", train_features.shape)
print("Test feature matrix shape:", test_features.shape)

np.save(f"{image_size}train_features.npy", train_features)
np.save(f"{image_size}train_labels.npy", train_labels)

np.save(f"{image_size}test_features.npy", test_features)
np.save(f"{image_size}test_labels.npy", test_labels)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import mrmr

#Load data from disk
image_size = "(224, 224)"
train_features = np.load(f"{image_size}train_features.npy")
train_labels   = np.load(f"{image_size}train_labels.npy")
test_features  = np.load(f"{image_size}test_features.npy")
test_labels    = np.load(f"{image_size}test_labels.npy")

#Most of these models prefer scaled values between 0 and 1, so performing that here
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_features)
test_scaled  = scaler.transform(test_features)

#Stores each of the datasets and their corrosponding number of components
datasets = {}
components = {}

datasets['Unreduced CNN Vector'] = (train_scaled, test_scaled)
components['Unreduced CNN Vector'] = 512;

#Creates the PCA dataset to account for 99% variance
pca = PCA(n_components=0.99, random_state=42)
train_pca = pca.fit_transform(train_scaled)
test_pca  = pca.transform(test_scaled)
datasets['PCA'] = (train_pca, test_pca)
components['PCA'] = train_pca.shape[1]

#Creates the LDA dataset
lda = LinearDiscriminantAnalysis()
train_lda = lda.fit_transform(train_scaled, train_labels)
test_lda  = lda.transform(test_scaled)
datasets['LDA'] = (train_lda, test_lda)
components['LDA'] = train_lda.shape[1]

#Creates the MI + PCA set
K = 300  # number of features to keep during MI. 300 worked the best through testing.
df_train = pd.DataFrame(train_scaled)
df_train['label'] = train_labels

#MI is supervised, so it needs to take in the labels when evaluating
selected_features = mrmr.mrmr_classif(
    X=df_train.drop('label', axis=1),
    y=df_train['label'],
    K=K
)
train_mrmr = train_scaled[:, selected_features]
test_mrmr  = test_scaled[:, selected_features]
#The PCA step in MI + PCA. Once again keeping 99% variance
pca_mrmr = PCA(n_components=0.99, random_state=42)
train_mrmr_pca = pca_mrmr.fit_transform(train_mrmr)
test_mrmr_pca  = pca_mrmr.transform(test_mrmr)
datasets['MI+PCA'] = (train_mrmr_pca, test_mrmr_pca)
components['MI+PCA'] = train_mrmr_pca.shape[1]

#The list of classifiers we want to use
classifier_list = {
    'Logistic Regression': LogisticRegression(max_iter=500), #500 performed the best for accuracy
    'SVM': SVC(kernel='rbf', C=50, gamma='scale'), #Using anything other than rbf and scale performed extremely poorly
    'XGBoost': XGBClassifier( #This is a relatively small XGBoost tree, but making it larger didn't help
        n_estimators=100, learning_rate=0.2, max_depth=6,
        subsample=0.8, colsample_bytree=0.8, random_state=42,
        use_label_encoder=False, eval_metric='logloss'
    ),
    'KNN': KNeighborsClassifier(n_neighbors=5) #k=5
}

#Store the accuracies. We'll need them later
accuracies = {key: [] for key in datasets.keys()}

#Iterates thorugh every dataset within the sets created earlier, and runs each classifier on them.
#Accuracies are stored for plotting later, and also the confusion matrices are printed
for dataset_name, (X_train, X_test) in datasets.items():
    for clf_name, clf in classifier_list.items():
        clf.fit(X_train, train_labels)
        acc = clf.score(X_test, test_labels)
        accuracies[dataset_name].append(acc)

        preds = clf.predict(X_test)

        cm = confusion_matrix(test_labels, preds)
        disp = ConfusionMatrixDisplay(cm)
        disp.plot(cmap='Blues')
        plt.title(dataset_name + " with " + clf_name + "Confusion Matrix")
        plt.show()


#I like these colors :) 
colors = ['skyblue', 'salmon', 'lightgreen', 'plum']  # for the datasets
control_accs = [0.80, 0.83]

#The controls are hardcoded in later
control_names = ['Control: Kaspersky', 'Control: ResNet18']
control_colors = ['gold', 'orange']

all_classifiers = list(classifier_list.keys()) + control_names
x = np.arange(len(all_classifiers))
width = 0.275

fig, ax = plt.subplots(figsize=(12,6))

#THIS CREATES A VERY UGLY PLOT BY DEFAULT. 
#It will plot every single classifier and dataset on the same chart, and everything smooshes into each other.
#If you want to examine an individual classifier, I reccomend commenting out / removing it from the classifier list. Everything looks horrible otherwise
for i, key in enumerate(datasets.keys()):
    ax.bar(x[:len(classifier_list)] + i*width - (len(datasets)-1)*width/2, 
           accuracies[key], width, label=key, color=colors[i])
    
    for j, acc in enumerate(accuracies[key]):
        #puts the num components on top of the bar
        ax.text(
            x=j + i*width - (len(datasets)-1)*width/2,
            y=0.75,  #hardcoded magic number. Puts the num of components nicely on the bar
            s=str(components[key]),
            ha='center',
            va='center',
            fontsize=9,
            color='black'
        )
        #Puts the accuracy above the bar
        ax.text(
            x=j + i*width - (len(datasets)-1)*width/2,
            y=acc + 0.005,
            s=f"{acc*100:.1f}%",
            ha='center',
            va='bottom',
            fontsize=9
        )

#This plots the hardcoded values for kaspersky and resnet
for i, (name, acc) in enumerate(zip(control_names, control_accs)):
    ax.bar(len(classifier_list) + i, acc, width, label=name, color=control_colors[i])
    ax.text(len(classifier_list) + i, acc + 0.005, f"{int(acc*100)}%", 
            ha='center', va='bottom', fontsize=9)

#X labels
ax.set_xticks(x)
ax.set_xticklabels(all_classifiers)

#Y labels and legend
ax.set_ylabel('Accuracy')
ax.set_xlabel('Classifier')
ax.set_title('Classifier Performance Across Different Dimension Reduction Methods')
ax.set_ylim(0.5, 1.05)
ax.grid(axis='y', linestyle='--', alpha=0.7)
ax.legend(title='Dataset', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

